### Covid 19 Confirmed Cases in Scotland 

* Data Loaded from https://github.com/watty62/Scot_covid19 
* Thanks to Ian Watt for putting it together 

In [10]:
# relevant libraries
# essential libraries
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from fbprophet import Prophet
import pycountry
import plotly.express as px
import plotly.io as pio
from functools import reduce 

In [11]:
# load files from github more details on https://github.com/watty62/Scot_covid19
dail_url = 'https://raw.githubusercontent.com/watty62/Scot_covid19/master/data/processed/new_daily_cases.csv'
nat_cases_url ='https://raw.githubusercontent.com/watty62/Scot_covid19/master/data/processed/regional_cases.csv'
scot_conf = pd.read_csv(dail_url)
nat_cases_df = pd.read_csv(nat_cases_url)
scot_conf.columns=['Date','Confirmed']
scot_conf.head()

,Date,Confirmed
0,02-Mar-2020,1
1,03-Mar-2020,0
2,04-Mar-2020,2
3,05-Mar-2020,3
4,06-Mar-2020,5


In [12]:
# add cummulative sum column 
scot_conf['Cum_sum'] = scot_conf['Confirmed'].cumsum()
scot_conf.tail()

,Date,Confirmed,Cum_sum
15,17-Mar-2020,24,195
16,18-Mar-2020,32,227
17,19-Mar-2020,39,266
18,20-Mar-2020,56,322
19,21-Mar-2020,51,373


#### Time-series Files 

In [13]:
nat_cases_df.drop('Grand Total', axis=1, inplace=True)
nat_cases_df.head()

,Date,Ayrshire and Arran,Borders,Dumfries and Galloway,Fife,Forth Valley,Grampian,Greater Glasgow and Clyde,Highland,Lanarkshire,Lothian,Orkney,Shetland,Tayside,Western Isles
0,01-Mar-2020,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,02-Mar-2020,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,03-Mar-2020,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,04-Mar-2020,1,0,0,0,0,1,0,0,0,0,0,0,1,0
4,05-Mar-2020,1,0,0,0,1,2,1,0,0,0,0,0,1,0


In [14]:
# tidy the data 
temp = nat_cases_df.transpose()
temp = temp.reset_index()

# Chagne names of the columns 

temp.columns = temp.iloc[0, 0:]
temp.rename(columns={'Date':'Region'}, inplace=True)

# drop first row (contains the names of the orginal df)
temp = temp.iloc[1:]
dates_list = temp.columns.to_list()[1:]

# tidy data df_confirmed 
tempM = pd.melt(temp, id_vars='Region',\
     value_vars=dates_list, var_name='Date', value_name='Confirmed')
tempM.head()

,Region,Date,Confirmed
0,Ayrshire and Arran,01-Mar-2020,0
1,Borders,01-Mar-2020,0
2,Dumfries and Galloway,01-Mar-2020,0
3,Fife,01-Mar-2020,0
4,Forth Valley,01-Mar-2020,0


#### Latest Results / Some Visualisations

In [15]:
# aggregate numbers by region 
cases_grouped = tempM.groupby('Region')['Confirmed'].max().reset_index()
cases_grouped = cases_grouped.sort_values(by='Confirmed', ascending=False)
cases_grouped = cases_grouped.reset_index(drop=True)
cases_grouped.style.background_gradient(cmap='Reds')

,Region,Confirmed
0,Greater Glasgow and Clyde,110
1,Lanarkshire,49
2,Lothian,44
3,Tayside,34
4,Forth Valley,27
5,Grampian,24
6,Shetland,24
7,Ayrshire and Arran,21
8,Dumfries and Galloway,13
9,Fife,13


In [16]:
# Cumulative numbers of confirmed cases in Scotland over time 

temp = tempM.groupby(['Date'])['Confirmed'].sum().reset_index().sort_values('Confirmed', ascending=True)
fig = px.bar(temp, x="Date", y="Confirmed", text='Confirmed',title='Confirmed Cases', height=600,
            template='plotly_white')
        
#fig.update_yaxes(type="log")
t_text = 'In Total ' + str(cases_grouped['Confirmed'].sum()) + ' Confirmed Cases in (Scotland)'
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(title_text=t_text, title_x=0.5)

In [17]:
# Per region
fig = px.bar_polar(cases_grouped, r="Confirmed", theta="Region", color="Region", template="plotly_dark",
                  color_discrete_sequence= px.colors.sequential.Plasma_r)
fig.show()

In [18]:
# Spread of the virus over time per region 

temp = tempM.groupby(['Date', 'Region'])['Confirmed'].sum().reset_index().sort_values('Confirmed', ascending=True)
fig = px.bar(temp, x="Date", y="Confirmed", color='Region', title='Confirmed Cases', height=600,
       facet_col='Region', facet_col_wrap=3,template='plotly_white')
        
#fig.update_yaxes(type="log")
t_text = str(cases_grouped['Confirmed'].sum()) + ' Confirmed Cases in (Scotland)'
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_layout(title_text=t_text, title_x=0.5)

#### To do 
* Add death cases 
* Add recovered 
* number of tests carried out